## Experiment description

The goal of this experiment is to train a TM classifier trained on 40% of the data available in Wind Farm C, and then evaluate the model on the remaining 60% of the data. The model will be evaluated on the accuracy score.

In [4]:
# Import all required libraries

import numpy as np
import pickle

from tmu.models.classification.vanilla_classifier import TMClassifier

In [5]:
train_datasets = np.array([34, 7])
test_datasets = np.array([53, 27])

In [6]:
## Helper functions for saving the model and accuracy

# Helper function to save the model
def save_model(model, filename):
    with open(filename, "wb") as f:
        pickle.dump(model, f)


# Helper function to save the accuracy
def save_accuracy(epoch, accuracy, tp, tn, fp, fn):
    with open("accuracy.txt", "a") as f:
        f.write(f"{epoch},{accuracy},{tp},{tn},{fp},{fn}\n")


# Helper function to load dataset
def load_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data_train/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_test_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data_test/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def get_train_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_dataset("B", dataset) for dataset in train_datasets])
    labels = np.concatenate([load_dataset_labels("B", dataset) for dataset in train_datasets])
    return dataset, labels


def get_test_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_test_dataset("B", dataset) for dataset in test_datasets])
    labels = np.concatenate([load_test_dataset_labels("B", dataset) for dataset in test_datasets])
    return dataset, labels

In [8]:
epochs = 100
platform = "CPU"

tm = TMClassifier(
    number_of_clauses=1000,
    T=1000,
    s=10.0,
    max_included_literals=32,
    weighted_clauses=True,
    platform=platform,
    batch_size=250,
)

best_accuracy = 0

print(f"Running {TMClassifier} for {epochs} epochs")

(X_train, y_train) = get_train_dataset()
(X_test, y_test) = get_test_dataset()

for epoch in range(epochs):

    tm.fit(X_train, y_train)

    pred = tm.predict(X_test)

    pred_0s = np.count_nonzero(pred == 0)
    pred_1s = np.count_nonzero(pred == 1)

    print(f"Predicted 0s: {pred_0s}, Predicted 1s: {pred_1s}")

    accuracy = np.sum(pred == y_test) / len(y_test)

    # the number of false positives based on g and p
    fp = np.sum((pred == 1) & (y_test == 0))

    # the number of true negatives based on g and p
    tn = np.sum((pred == 0) & (y_test == 0))

    # the number of true positives based on g and p
    tp = np.sum((pred == 1) & (y_test == 1))

    # the number of false negatives based on g and p
    fn = np.sum((pred == 0) & (y_test == 1))

    acc = tn / (fp + tn)

    print(f"Accuracy = {accuracy:.5f}, TP = {tp}, TN = {tn}, FP = {fp}, FN = {fn}")

    save_accuracy(epoch, acc, tp, tn, fp, fn)

    # Print every 20 epochs
    #if (epoch + 1) % 20 == 0:
    print(f"Epoch: {epoch + 1}, Accuracy: {acc:.5f}")

    if acc > best_accuracy:
        best_accuracy = acc

        print("Saving model")
        save_model(tm, "best.pkl")


Running <class 'tmu.models.classification.vanilla_classifier.TMClassifier'> for 100 epochs
Predicted 0s: 93079, Predicted 1s: 27788
Accuracy = 0.72255, TP = 4543, TN = 82789, FP = 23245, FN = 10290
Epoch: 1, Accuracy: 0.78078
Saving model
Predicted 0s: 99176, Predicted 1s: 21691
Accuracy = 0.75264, TP = 3313, TN = 87656, FP = 18378, FN = 11520
Epoch: 2, Accuracy: 0.82668
Saving model
Predicted 0s: 99020, Predicted 1s: 21847
Accuracy = 0.76089, TP = 3890, TN = 88077, FP = 17957, FN = 10943
Epoch: 3, Accuracy: 0.83065
Saving model
Predicted 0s: 97172, Predicted 1s: 23695
Accuracy = 0.75270, TP = 4319, TN = 86658, FP = 19376, FN = 10514
Epoch: 4, Accuracy: 0.81727
Predicted 0s: 100223, Predicted 1s: 20644
Accuracy = 0.76347, TP = 3444, TN = 88834, FP = 17200, FN = 11389
Epoch: 5, Accuracy: 0.83779
Saving model
Predicted 0s: 100991, Predicted 1s: 19876
Accuracy = 0.77133, TP = 3535, TN = 89693, FP = 16341, FN = 11298
Epoch: 6, Accuracy: 0.84589
Saving model
Predicted 0s: 98748, Predicted 1